In [1]:
# !pip install opencv-python
# !pip install qrcode

In [2]:
# import libraries
import requests
import json
import os
from datetime import date
from web3.providers.rpc import HTTPProvider
from web3.contract import ConciseContract
import cv2 as cv
import openpyxl
import qrcode
from dotenv import load_dotenv
from pathlib import Path
from web3 import Web3
from pprint import pprint
load_dotenv()

ModuleNotFoundError: No module named 'web3'

In [ ]:
# assign values to keys and addresses
pin_url = 'https://api.pinata.cloud/pinning/pinJSONToIPFS'
PINATA_API_KEY = os.getenv('PINATA_API_KEY')
PINATA_SECRET_KEY = os.getenv('PINATA_SECRET_KEY')
WEB3_PROVIDER_URI = 'HTTP://127.0.0.1:8545'
w3 = Web3(Web3.HTTPProvider(WEB3_PROVIDER_URI))
CHAINVAX_ADDRESS = '0xcEfBD2E908C32A31A894Ca88e8625fd53256ACe0'
headers = {
    "Content-Type": "application/json",
    "pinata_api_key": PINATA_API_KEY,
    "pinata_secret_api_key": PINATA_SECRET_KEY,
}

In [ ]:
#We are going to make a function that sends a json object to pinata, so they can host it on the IPFS for us

def pinJSONtoIPFS(json):
    r = requests.post(pin_url, data=json, headers=headers)
    ipfs_hash = r.json()["IpfsHash"]
    return f"https://gateway.pinata.cloud/ipfs/{ipfs_hash}"

In [ ]:
#We are going to make a function that sends a IMG object to pinata, so they can host it on the IPFS for us
pin_img_url = 'https://api.pinata.cloud/pinning/pinFileToIPFS'
img_headers = {
 "pinata_api_key": PINATA_API_KEY,
 "pinata_secret_api_key": PINATA_SECRET_KEY,
}
def pinIMGtoIPFS(img_pth):
    file = {'file': open(img_pth, 'rb')}
    r = requests.post(pin_img_url,files = file,headers=img_headers)
    ipfs_hash = r.json()["IpfsHash"]
    return f"https://gateway.pinata.cloud/ipfs/{ipfs_hash}"

In [ ]:
def convertDataToJSON(name, dob, serial, date, docNo):
    data = {
    "pinataOptions": {"cidVersion": 1},
    "pinataContent": {
    "certificate": "COVID Vaccination Certificate",
    "name": name,
    "date of birth": dob,
    "vaccine serial number": serial,
    "vaccine date": date,
    "doccument number": docNo,}
}
    return data #was doing json.dumps(data) but moved the .dumps to createVaccineCertificate function

In [ ]:
# Inputs for the person being vaccinated to create a digital certificate as an NFT and send to their Ethernet wallet
name = input("what is your name?: ")
dob = input("what is your date of birth?: ")
valid_from = str(date.today())
serial = input("what is the vaccine serial number?: ")
vax1 = input("what is your first vaccine dose: ")
vax2 = input("what is your second vaccine dose?: ")
vaxType = input("Vaccine Type: ")
vaccinationDate = valid_from
vaccinationType = vaxType
vaccineSerialNumber = serial
vax_serial_number = serial
todays_date = valid_from
address_owner = '0xc4026624e88D1fdC6E53cf901b22Dff30423F2ee'#this will be the owner of the smart contract
address_recipient = input("what is your Ethernet wallet address?: ")

In [ ]:
# what is your name?:  ChainVax Test
# what is your date of birth?:  01/01/0101
# what is the vaccine serial number?:  XPZR0000123789
# what is your first vaccine dose:  C19 Vax PZ - 01/01/21
# what is your second vaccine dose?:  C19 Vax PZ - 01/04/21
# Vaccine Type:  Pfizer
# what is your Ethernet wallet address?:  0x89CB4ACe78a8968abE2d668C1F9c5a699C35fB76

In [ ]:
# create function to interact with smart contract and prepare values for writing the certificate
def initContract():
    with open(Path("ChainVax.json")) as json_file:
        abi = json.load(json_file)
    return w3.eth.contract(address=CHAINVAX_ADDRESS, abi=abi)
chainvax = initContract()
reader = ConciseContract(chainvax)
docNo = reader.totalSupply() +1
vax_certi_temple_path = 'VaxCertiTemple.png'
output_path = './'
font_size = 1
font_color = (0,0,0)

In [ ]:
# this function creates a certificate from the arguments
def vax_cert (name, dob, valid_from, vax_serial_number, vax1, vax2):
    
    # read the certificate template
    vax_img = cv.imread(vax_certi_temple_path)

    # choose the font from opencv
    font = cv.FONT_HERSHEY_DUPLEX              

    # get the size of the name to be
    # printed
    text_size = cv.getTextSize(name,font, font_size, 4)[0]     

    cv.putText(vax_img, name,(20,252),font,font_size,font_color,1)
    cv.putText(vax_img, dob,(50,310),font,font_size,font_color,1)
    cv.putText(vax_img, valid_from,(50,373),font,font_size,font_color,1)
    cv.putText(vax_img, str(vax_serial_number),(20,490),font,font_size,font_color,1)
    cv.putText(vax_img, vax1,(10,545),font,font_size,font_color,1)
    cv.putText(vax_img, vax2,(10,570),font,font_size,font_color,1)

    # Output path along with the name of the
    # certificate generated
    certi_path = output_path + '/VaxCert2' + '.png'

    # Save the certificate                      
    cv.imwrite(certi_path,vax_img)
    
    return certi_path

In [ ]:
# this funciton also takes the inputs from cell [7] and uses the vax_cert function creating a certificate and uploading to the IPFS
# it then appends the IPFS URI to the json_data and uploads the json data to the IPFS
# it then creates a QR code of the ceritifacte image for easy visualisation

def createVaccineCertificate(name, dob, serial, todays_date, docNo, vax1, vax2):
    reader = ConciseContract(chainvax)
    docNo = reader.totalSupply() + 1
    json_data = convertDataToJSON(name, dob, serial, todays_date, docNo)
    png_file_path = vax_cert(name, dob, todays_date, docNo, vax1, vax2)
    img_ipfs = pinIMGtoIPFS(png_file_path)
    json_data["pinataContent"]['img_ipfs_uri'] = img_ipfs
    report_uri = pinJSONtoIPFS(json.dumps(json_data))
    
    qr = qrcode.QRCode(
    # integer from 1 to 40 that controls the size of the QR Code
    version=1,
    # error_correction parameter controls the error correction used for the QR Code
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    # box_size parameter controls how many pixels each "box" of the QR code is.
    box_size=10,   
    # border parameter controls how many boxes thick the border should be.
    # the default is 4, which is the minimum
    border=4,
    )
    # add your text here
    ##qr.add_data(pinIMGtoIPFS('VaxCert2.png'))
    qr.add_data(img_ipfs)
    qr.make(fit=True)
    # change the colors of the QR code
    img = qr.make_image(fill_color="green", back_color="Beige")
    # name the file and save it
    img.save("QR_VaxCert.png")
    return report_uri

In [ ]:
createVaccineCertificate(name, dob, serial, valid_from, docNo, vax1, vax2)

In [ ]:
# #test function to interact with smart contract
# def testFunction():
#     tx_hash = chainvax.functions.testFunction().transact({"from": w3.eth.accounts[0]})
#     receipt = w3.eth.waitForTransactionReceipt(tx_hash)
#     return receipt

In [ ]:
#testFunction()

In [ ]:
# these are the arguments for the solidity smart contract(address owner, string memory DOB,string memory NAME, string memory vaccinationType, string memory vaccinationDate, string memory vaccineSerialNumber, string memory token_uri, address recipient) public returns(uint)
# this funciton will now register the Vaccine Certificate on the blockchain

def registerVaccineDose(address_owner, dob, name, vaccinationType, vaccinationDate, vaccineSerialNumber, report_uri, address_recipient):
    tx_hash = chainvax.functions.registerVaccineDose(address_owner, dob, name, vaccinationType, vaccinationDate, vaccineSerialNumber, report_uri, address_recipient).transact({"from": w3.eth.accounts[0]})
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    return receipt
# This is calling a solidity function from python 
report_uri = createVaccineCertificate(name, dob, serial, todays_date, docNo, vax1, vax2)
receipt = registerVaccineDose(address_owner, dob, name, vaccinationType, vaccinationDate, vaccineSerialNumber, report_uri, address_recipient)
print(receipt)
print("Report IPFS Hash:", report_uri)